In [1]:
import datetime as dt
import sqlalchemy as sa
import os
import sys
import pandas as pd
import yfinance as yf
import time

In [2]:
username = os.getlogin()
external_folder_path = 'C:/Users/' + username + '/Documents/Projects/Financial_Securities/Custom_Python_Functions/'
sys.path.append(external_folder_path)
from custom_python_functions import create_connection, clear_table, load_key, decrypt, get_dates_for_years

key1 = 'user_key.ky'
key_file1 = 'user_key.txt'
key2 = 'pass_key.ky'
key_file2 = 'pass_key.txt'

key1 = load_key(f_path, key1)
uid = decrypt(f_path, key_file1, key1)

key2 = load_key(f_path, key2)
passwd = decrypt(f_path, key_file2, key2)

# Setup connection parameters
server = 'danvuk.database.windows.net'
dbase = 'Financial_Securities'

# Create a connection to the database
s, e = create_connection(server, dbase, uid, passwd)
s1 = s()  # Instantiate a session object

In [3]:
Base = sa.orm.declarative_base()

class Data_STG(Base):
    
    """
    SQLAlchemy ORM class representing the 'Data_STG' table in the 'Equities' schema.

    Attributes:
    __tablename__ (str): The name of the table in the database.
    __table_args__ (dict): Additional arguments for the table, including schema name.
    Date (Column): The date column, used as part of the primary key.
    Description (Column): The description column, used as part of the primary key.
    Float_Value1 (Column): A column for storing floating-point values, such as stock prices or metrics.
    Float_Value2 (Column): Another column for storing floating-point values.
    Float_Value3 (Column): Another column for storing floating-point values.
    Float_Value4 (Column): Another column for storing floating-point values.
    Int_Value1 (Column): A column for storing integer values, such as volumes or counts.
    """
    
    __tablename__ = 'Data_STG'
    __table_args__ = {'schema': 'Equities'}
    Date = sa.Column(sa.Date, primary_key=True)
    Description = sa.Column(sa.String, primary_key=True)
    Float_Value1 = sa.Column('Float_Value1', sa.Float)
    Float_Value2 = sa.Column('Float_Value2', sa.Float)
    Float_Value3 = sa.Column('Float_Value3', sa.Float)
    Float_Value4 = sa.Column('Float_Value4', sa.Float)
    Int_Value1 = sa.Column('Int_Value1', sa.BigInteger)  

In [4]:
# Clear the existing data in the Data_STG table
clear_table(s1, 'Financial_Securities.Equities.Data_STG')

In [5]:
# Define SQL query to retrieve tickers from the Equities table
sql_stat = """SELECT
  TRIM(Ticker) AS Ticker
  FROM [Financial_Securities].[Equities].[Equities]
  ORDER BY Ticker"""

try:
    # Execute the SQL query and read the results into a DataFrame
    df_tickers = pd.read_sql(sql_stat, s1.bind)
    ticker_list = df_tickers['Ticker'].values.tolist()
    
except sa.exc.SQLAlchemyError as e:
    # Handle exceptions during SQL query execution
    print(f"Issue querying Equities database table! Error: {e}")
    s1.close()
    raise

In [7]:
# Generate the date range of 3 years back as of yesterday
start_date, end_date = get_dates_for_years(3, 0)

In [8]:
def create_daily_pricing(ticker, start_date, end_date):
    
    """
    Retrieves and processes daily pricing data for a given ticker symbol.

    Args:
        ticker (str): The stock ticker symbol.
        start_date (str): The start date for the data retrieval.
        end_date (str): The end date for the data retrieval.

    Returns:
        a DataFrame with daily pricing data.
    """
    
    # Download stock data from Yahoo Finance
    stock_data = yf.download(ticker.replace(".", "-"), start=start_date, end=end_date)
    
    # Copy the data for processing
    df_tmp = stock_data.copy()
    df_tmp['Ticker'] = ticker
    df_tmp.reset_index(inplace=True)
    df_tmp['Date'] = pd.to_datetime(df_tmp['Date'], format='%Y-%m-%d')
    
    # Fill missing values using forward fill method
    df_tmp['Open'] = df_tmp['Open'].fillna(method="ffill")
    df_tmp['High'] = df_tmp['High'].fillna(method="ffill")
    df_tmp['Low'] = df_tmp['Low'].fillna(method="ffill")
    
    # Adjust prices and calculate the factor for adjustments
    df_tmp['Factor'] = df_tmp['Close'] / df_tmp['Adj Close']
    df_tmp['Close'] = round(df_tmp['Adj Close'].fillna(method="ffill"), 2)
    df_tmp['Factor'] = df_tmp['Close'] / df_tmp['Adj Close']
    df_tmp['Open'] = round(df_tmp['Open'] / df_tmp['Factor'], 2)
    df_tmp['High'] = round(df_tmp['High'] / df_tmp['Factor'], 2)
    df_tmp['Low'] = round(df_tmp['Low'] / df_tmp['Factor'], 2)
    
    # Select relevant columns and prepare DataFrame for database insertion
    df_tmp = df_tmp[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    df_tmp['Date2'] = df_tmp['Date']
    df_tmp.set_index(['Date2'], inplace=True)
    df_tmp.sort_values(by=['Date'], inplace=True)

    return df_tmp

In [9]:
# Initialize variables for processing
first_ticker = True


for ticker in ticker_list:
    # Fetch and process daily pricing data for each ticker
    df_tmp = create_daily_pricing(ticker, start_date, end_date)
    
    if len(df_tmp) > 0:
        # Combine data for all tickers into a single DataFrame
        if first_ticker:
            df_equities = df_tmp.copy()
            first_ticker = False
        else:
            df_equities = pd.concat([df_equities, df_tmp])
        
        # Sleep to avoid hitting API rate limits
        time.sleep(1)
        
print("Pricing data fetch is complete")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [16]:
if len(df_equities['Ticker'].drop_duplicates()) < len(ticker_list):
    print(f"Only {len(df_equities['Ticker'].drop_duplicates())} records out of {len(ticker_list)} records were fetched!")
else:
    print(f"All {len(ticker_list)} records were fetched!")

All 503 records were fetched!


In [14]:
# Insert the data into the Data_STG table
for index, row in df_equities.iterrows():
    try:
        q1 = Data_STG(
            Date=row['Date'],
            Description=row['Ticker'],
            Float_Value1=row['Open'],
            Float_Value2=row['High'],
            Float_Value3=row['Low'],
            Float_Value4=row['Close'],
            Int_Value1=row['Volume'],
        )
        s1.add(q1)
        
    except sa.exc.SQLAlchemyError as e:
        # Handle exceptions during data insertion
        message = f"Issue with updating Data_STG database table for Ticker: {row['Ticker']}. Error: {e}"
        print(message)
        s1.close()
        raise

# Commit all changes to the database
s1.commit()

print("Database data load is complete")


In [17]:
# SQL query to count the number of records in the Data_STG table
sql_stat = """SELECT COUNT(*) FROM [Financial_Securities].[Equities].[Data_STG]"""

try: 
    result = e.execute(sql_stat)  # Execute the count query
    cnt_recs = result.scalar()  # Get the count of records
    
# Handle SQLAlchemy errors if they occur during query execution
except sa.exc.SQLAlchemyError as e:
    # Print the error
    print(f"Issue querying Data_STG database table for count! Error: {e}")

        
# Compare the record counts and print the result    
if cnt_recs < len(df_equities):
    print(f"Only {cnt_recs} records out of {len(df_equities)} records were loaded into the Data_STG table!")
else:
    print(f"All {cnt_recs} records were loaded into the Data_STG table!")

All 465762 records were loaded into the Data_STG table!


In [ ]:
s1.close()  # Close the session